In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm, zscore, t, ttest_ind, permutation_test
from statsmodels.stats.weightstats import ztest


sns.set_theme(
    style='whitegrid',       # 배경 스타일 (grid 포함)
    font='BM Hanna Pro',     # 폰트
    rc={
        'axes.unicode_minus': False,  # 마이너스 기호 깨짐 방지
        'figure.figsize': (12, 6),    # 그래프 크기 설정
        'axes.labelsize': 14,         # 축 레이블 크기
        'xtick.labelsize': 12,        # x축 눈금 크기
        'ytick.labelsize': 12,        # y축 눈금 크기
        'legend.fontsize': 12,        # 범례 폰트 크기
        'axes.titleweight': 'bold',   # 제목 굵게
        'axes.titlesize': 16          # 제목 크기
    }
)

%config InlineBackend.figure_format = 'retina'

In [3]:
data = pd.read_csv("ns_book7.csv", low_memory=False)

data.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,인공지능과 흙,김동훈 지음,민음사,2021,9788937444319,NaN,NaN,NaN,NaN,1,0,2021-03-19
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,2021,9791190123969,NaN,NaN,NaN,NaN,1,0,2021-03-19
2,3,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,블랙피쉬,2021,9788968332982,NaN,NaN,NaN,NaN,1,0,2021-03-19
3,4,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",문학세계사,2021,9788970759906,NaN,NaN,NaN,NaN,1,0,2021-03-19
4,5,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,김영사,2021,9788934990833,NaN,NaN,NaN,NaN,1,0,2021-03-19


In [4]:
python_book_idx = data['주제분류번호'].str.startswith('00') & data['도서명'].str.contains('파이썬')
python_books = data[python_book_idx]

cplus_book_idx = data['주제분류번호'].str.startswith('00') & data['도서명'].str.contains('C++', regex=False)
cplus_books = data[cplus_book_idx]

java_book_idx = data['주제분류번호'].str.startswith('00') & data['도서명'].str.contains('자바스크립트')
java_books = data[java_book_idx]

In [5]:
python_mean = np.mean(python_books['대출건수'])
python_std = np.std(python_books['대출건수'])
python_len = len(python_books)

cplus_mean = np.mean(cplus_books['대출건수'])
cplus_std = np.std(cplus_books['대출건수'])
cplus_len = len(cplus_books['대출건수'])

java_mean = np.mean(java_books['대출건수'])
java_std = np.std(java_books['대출건수'])
java_len = len(java_books['대출건수'])

print(python_mean, python_std, python_len)
print(cplus_mean, cplus_std, cplus_len)

14.749003984063744 12.740309535319817 251
11.595505617977528 9.19662749754605 89


In [6]:
z1 = (python_mean - cplus_mean) / np.sqrt(((python_std ** 2) / python_len) + ((cplus_std ** 2) / cplus_len))

cdf_z1 = norm.cdf(z1)

p1_value = (1 - cdf_z1) * 2

print(z1, p1_value)

2.4954081951407083 0.012581230643561092


In [7]:
z2, p2_value = ztest(python_books['대출건수'], cplus_books['대출건수'])

print(z2, p2_value)

2.1390005694958574 0.03243562225015445


In [8]:
t, p3_value = ttest_ind(python_books['대출건수'], cplus_books['대출건수'])

print(t, p3_value)

2.1390005694958574 0.033151795202247834


In [9]:
def statistic(x, y):
    return np.mean(x) - np.mean(y)

res = permutation_test((python_books['대출건수'], cplus_books['대출건수']), statistic, random_state=42)
print(res.statistic, res.pvalue)

3.1534983660862164 0.0258


In [10]:
res1 = permutation_test(
                        (python_books['대출건수'], cplus_books['대출건수']), 
                        lambda x, y: np.mean(x) - np.mean(y),
                        random_state=42
                        )

print(res1.statistic, res1.pvalue)

3.1534983660862164 0.0258


In [12]:
res2 = permutation_test(
    (python_books['대출건수'], java_books['대출건수']),
    statistic,
    random_state=42
)

print(res2.statistic, res2.pvalue)

-0.7843293492695889 0.57


In [13]:
se1 = np.sqrt(((python_std ** 2) / python_len) + ((java_std ** 2) / java_len))

z5 = (python_mean - java_mean) / se1

print(z5)

-0.5940069858292665


In [ ]:
norm.cdf(z5)

np.float64(0.27625372028851186)

In [22]:
p4_value = (1 - norm.cdf(abs(z5))) * 2

print(p4_value)

0.5525074405770236


In [23]:
ztest(python_books['대출건수'], java_books['대출건수'])

(np.float64(-0.5523987093630017), np.float64(0.5806752146816628))

In [24]:
t, p5_value = ttest_ind(python_books['대출건수'], java_books['대출건수'])

print(t, p5_value)

-0.5523987093630017 0.5810238142528562


In [25]:
from scipy.stats import ttest_ind

# 데이터 설정
mean_A = 200000
mean_B = 195000
std_A = 25000
std_B = 20000
n_A = 400
n_B = 300

# 가상의 정규분포 데이터 생성 (평균과 표준편차 반영)
import numpy as np
np.random.seed(42)  # 재현성을 위해 시드 고정

sample_A = np.random.normal(mean_A, std_A, n_A)
sample_B = np.random.normal(mean_B, std_B, n_B)

# 독립 표본 t-검정 수행
t_stat, p_value = ttest_ind(sample_A, sample_B, equal_var=False)  # Welch's t-test

# 결과 출력
print(f"T-Statistic: {t_stat}")
print(f"p-value: {p_value}")

# 가설 검정
if p_value < 0.05:
    print("귀무가설 기각: 두 그룹의 평균 차이는 유의미함!")
else:
    print("귀무가설 유지: 두 그룹의 평균 차이는 통계적으로 유의미하지 않음.")

T-Statistic: 3.894701773421466
p-value: 0.00010791791869268736
귀무가설 기각: 두 그룹의 평균 차이는 유의미함!
